<a href="https://colab.research.google.com/github/JasonLn0711/1131_TAICA_ML/blob/main/FP__stage_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install minisom

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.3-py3-none-any.whl size=11706 sha256=3656e7961cdf75517c6d22d1cfa4c8a1e3392ae8d4f33279267cd41931b1b9c8
  Stored in directory: /root/.cache/pip/wheels/96/98/a5/52dee3e8ed1dbfc4d77e4da41b6d89dd7ab9ead1b921e766f8
Successfully built minisom


In [4]:
# Unsupervised Evaluation
# Method 4: Self-Organizing Maps (SOM)

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, zero_one_loss
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.arima.model import ARIMA
from minisom import MiniSom # Fix v4.1.2: Instead of importing 'somoclu', we use 'minisom' to handle Self-Organizing Maps (SOM) for fixing compatibility issues with mac OS due to missing linked libraries.

In [5]:
from google.colab import drive

# Load data
drive.mount('/d')
file_path_1 = '/d/MyDrive/train_data.csv'
file_path_2 = '/d/MyDrive/same_season_test_data.csv'

train_data = pd.read_csv(file_path_1)
test_data = pd.read_csv(file_path_2)

Mounted at /d


In [6]:
# Update the target variable for test data using Self-Organizing Maps (SOM)
# Select Relevant Features
features = [
    'home_team_wins_mean',
    'away_team_wins_mean',
    'home_batting_batting_avg_mean',
    'away_batting_batting_avg_mean'
]
X = test_data[features].fillna(0)

# Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SOM using MiniSom
som = MiniSom(x=10, y=10, input_len=X_scaled.shape[1], sigma=1.0, learning_rate=0.5)
som.random_weights_init(X_scaled)
som.train_random(X_scaled, 100)  # Train for 100 iterations

# Extract Cluster Assignments
bmus = np.array([som.winner(x) for x in X_scaled])
test_data['cluster'] = bmus[:, 0] * 10 + bmus[:, 1]

# Assign Proxy Target Variable
# Determine which cluster is more likely to be home wins
cluster_home_win = test_data.groupby('cluster')['home_team_wins_mean'].mean().idxmax()
test_data['home_team_win'] = (test_data['cluster'] == cluster_home_win).astype(int)

In [7]:
# Drop date columns from the datasets
train_data.drop(columns=['date'], inplace=True, errors='ignore')

# Identify categorical columns
categorical_cols = ['home_team_abbr', 'away_team_abbr', 'home_pitcher', 'away_pitcher', 'home_team_season', 'away_team_season']
numeric_cols = [col for col in train_data.columns if col not in categorical_cols + ['home_team_win']]

# Creating pipelines for the preprocessing of numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Column transformer for applying different preprocessing to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [8]:
# Split features and target
X_train = train_data.drop(columns=['home_team_win'])
y_train = train_data['home_team_win']
X_test = test_data.drop(columns=['home_team_win'])
y_test = test_data['home_team_win']

# Apply preprocessing
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [9]:
# Step 1: Outlier Detection with Modified Threshold
def christoffel_darboux_kernel_outliers(data, n_neighbors=5, threshold=7.0):
    nn = NearestNeighbors(n_neighbors=n_neighbors).fit(data)
    distances, _ = nn.kneighbors(data)
    kernel_density_scores = np.mean(distances, axis=1)

    outliers = np.where(kernel_density_scores > threshold)[0]
    return outliers

# Detect and remove outliers from training data
outliers = christoffel_darboux_kernel_outliers(X_train_preprocessed)
print(f"Initial Outliers Detected: {len(outliers)}")

# Adjust outlier removal to prevent empty dataset
if len(outliers) < X_train_preprocessed.shape[0] * 0.8:  # Keep at least 20% of the data
    X_train_filtered = X_train_preprocessed[np.isin(range(len(X_train)), outliers, invert=True)]
    y_train_filtered = y_train.drop(outliers).reset_index(drop=True)
else:
    print("Too many outliers detected; skipping outlier removal.")
    X_train_filtered = X_train_preprocessed
    y_train_filtered = y_train

Initial Outliers Detected: 7919


In [10]:
# Step 2: Feature Engineering with ARMA for Temporal Dependencies
win_ratios = train_data.groupby('home_team_abbr')['home_team_win'].apply(list)
forecasted_ratios = {}
for team, ratios in win_ratios.items():
    if len(ratios) > 5:
        try:
            model = ARIMA(ratios, order=(1, 0, 1))  # Simplified ARIMA model
            arma_model = model.fit()
            forecast = arma_model.forecast(steps=1)
            forecasted_ratios[team] = forecast[0]
        except Exception as e:
            print(f"ARIMA model failed for team {team}: {e}")

train_data['forecasted_win_ratio'] = train_data['home_team_abbr'].map(forecasted_ratios).fillna(0)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [11]:
# Step 3: Baseline Models - Logistic Regression and Decision Tree
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_filtered, y_train_filtered)
logreg_pred = logreg.predict(X_test_preprocessed)
print("Logistic Regression - Accuracy:", accuracy_score(y_test, logreg_pred))

dt = DecisionTreeClassifier()
dt.fit(X_train_filtered, y_train_filtered)
dt_pred = dt.predict(X_test_preprocessed)
print("Decision Tree - Accuracy:", accuracy_score(y_test, dt_pred))

Logistic Regression - Accuracy: 0.49700889248181085
Decision Tree - Accuracy: 0.48536782538399353


In [12]:
# Step 4: Custom Online Learning Model - Mondrian Forest Approximation with Incremental Updates
forest = RandomForestClassifier(n_estimators=10, random_state=42)
forest.fit(X_train_filtered, y_train_filtered)

# Function to incrementally update the model
def incremental_update(model, new_data, new_labels):
    model.fit(new_data, new_labels)
    return model

# Initial evaluation
forest_pred = forest.predict(X_test_preprocessed)
print("Initial Mondrian Forest Approximation - Accuracy:", accuracy_score(y_test, forest_pred))

Initial Mondrian Forest Approximation - Accuracy: 0.5219078415521423


In [13]:
# Step 5: Simulate Incremental Learning with Time-Based Data Batches
batch_size = 50  # Define batch size for incremental updates
num_batches = X_test_preprocessed.shape[0] // batch_size

for i in range(num_batches):
    start = i * batch_size
    end = (i + 1) * batch_size
    X_batch = X_test_preprocessed[start:end]
    y_batch = y_test.iloc[start:end]

    # Evaluate before update
    batch_pred = forest.predict(X_batch)
    batch_accuracy = accuracy_score(y_batch, batch_pred)
    print(f"Batch {i+1} Accuracy before update: {batch_accuracy:.2f}")

    # Incremental update
    forest = incremental_update(forest, X_batch, y_batch)

    # Evaluate after update
    batch_pred_updated = forest.predict(X_batch)
    batch_accuracy_updated = accuracy_score(y_batch, batch_pred_updated)
    print(f"Batch {i+1} Accuracy after update: {batch_accuracy_updated:.2f}")

Batch 1 Accuracy before update: 0.50
Batch 1 Accuracy after update: 0.98
Batch 2 Accuracy before update: 1.00
Batch 2 Accuracy after update: 1.00
Batch 3 Accuracy before update: 1.00
Batch 3 Accuracy after update: 1.00
Batch 4 Accuracy before update: 0.96
Batch 4 Accuracy after update: 0.98
Batch 5 Accuracy before update: 1.00
Batch 5 Accuracy after update: 1.00
Batch 6 Accuracy before update: 1.00
Batch 6 Accuracy after update: 1.00
Batch 7 Accuracy before update: 1.00
Batch 7 Accuracy after update: 1.00
Batch 8 Accuracy before update: 1.00
Batch 8 Accuracy after update: 1.00
Batch 9 Accuracy before update: 1.00
Batch 9 Accuracy after update: 1.00
Batch 10 Accuracy before update: 0.94
Batch 10 Accuracy after update: 1.00
Batch 11 Accuracy before update: 1.00
Batch 11 Accuracy after update: 1.00
Batch 12 Accuracy before update: 1.00
Batch 12 Accuracy after update: 1.00
Batch 13 Accuracy before update: 1.00
Batch 13 Accuracy after update: 1.00
Batch 14 Accuracy before update: 1.00
Batch

In [14]:
# Step 6: Model Evaluation - Accuracy and 0/1 Error
models = {
    "Logistic Regression": logreg,
    "Decision Tree": dt,
    "Mondrian Forest Approximation": forest
}

for model_name, model in models.items():
    y_pred = model.predict(X_test_preprocessed)
    accuracy = accuracy_score(y_test, y_pred)
    error = zero_one_loss(y_test, y_pred)
    print(f"{model_name} - Accuracy: {accuracy:.2f}, 0/1 Error: {error:.2f}")

Logistic Regression - Accuracy: 0.50, 0/1 Error: 0.50
Decision Tree - Accuracy: 0.49, 0/1 Error: 0.51
Mondrian Forest Approximation - Accuracy: 0.99, 0/1 Error: 0.01
